In [1]:
# https://github.com/backstopmedia/tensorflowbook

In [2]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once('~/pooh/poohdrop/mldata/tf_inteligence/output/training-images/*.tfrecords'))
reader = tf.TFRecordReader()
_, serialized = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'image': tf.FixedLenFeature([], tf.string),
    })
record_image = tf.decode_raw(features['image'], tf.uint8)
record_image

<tf.Tensor 'DecodeRaw:0' shape=(?,) dtype=uint8>

In [3]:
image = tf.reshape(record_image, [250, 151, 1])
label = tf.cast(features['label'], tf.string)
min_after_dequeue = 10
batch_size = 3
capacity = min_after_dequeue + 3 * batch_size
image_batch, label_batch = tf.train.shuffle_batch(
    [image, label], batch_size=batch_size, capacity=capacity,
    min_after_dequeue=min_after_dequeue)
image_batch

<tf.Tensor 'shuffle_batch:0' shape=(3, 250, 151, 1) dtype=uint8>

In [4]:
# Converting the images to a float of [0,1) to match the expected input to convolution2d
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)
conv2d_layer_one = tf.contrib.layers.convolution2d(
    float_image_batch,
    num_outputs=32,
    kernel_size=(5,5),
    activation_fn=tf.nn.relu,
    weights_initializer=tf.random_normal_initializer(),
    stride=(2,2),
    trainable=True)
pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
                                ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding='SAME')

In [5]:
# Note, the first and last dimension of the convolution output hasn't changed but the
# middle two dimensions have
conv2d_layer_one.get_shape(), pool_layer_one.get_shape()

(TensorShape([Dimension(3), Dimension(125), Dimension(76), Dimension(32)]),
 TensorShape([Dimension(3), Dimension(63), Dimension(38), Dimension(32)]))

In [6]:
conv2d_layer_two = tf.contrib.layers.convolution2d(
    pool_layer_one,
    num_outputs=64,
    kernel_size=(5,5),
    activation_fn=tf.nn.relu,
    weights_initializer=tf.random_normal_initializer(),
    stride=(1,1),
    trainable=True)
pool_layer_two = tf.nn.max_pool(conv2d_layer_two,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding='SAME')
conv2d_layer_two.get_shape(), pool_layer_two.get_shape()



(TensorShape([Dimension(3), Dimension(63), Dimension(38), Dimension(64)]),
 TensorShape([Dimension(3), Dimension(32), Dimension(19), Dimension(64)]))

In [7]:
flattened_layer_two = tf.reshape(
    pool_layer_two,
    [batch_size, -1])
flattened_layer_two.get_shape()

TensorShape([Dimension(3), Dimension(38912)])

In [8]:
hidden_layer_three = tf.contrib.layers.fully_connected(
    flattened_layer_two,
    512,
    weights_initializer=lambda i, dtype, partition_info=None:tf.truncated_normal([38912, 512], stddev=0.1),
    activation_fn=tf.nn.relu)
hidden_layer_three = tf.nn.dropout(hidden_layer_three, 0.1)

final_fully_connected = tf.contrib.layers.fully_connected(
    hidden_layer_three,
    120,
    weights_initializer=lambda i, dtype, partition_info=None: tf.truncated_normal([512, 120], stddev=0.1))

import glob
labels = list(map(lambda c: c.split("/")[-1], glob.glob("/Users/user/Dropbox/poohdrop/mldata/data-tf_inteligence/imagenet-dogs/*")))
labels[:10]
train_labels = tf.map_fn(lambda l: tf.where(tf.equal(labels, l))[0,0:1][0], label_batch, dtype=tf.int64)

train_labels[:10]
type(label_batch)

tensorflow.python.framework.ops.Tensor

In [9]:
# setup-only-ignore
loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=final_fully_connected, labels=train_labels))
batch = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
    0.01,
    batch * 3,
    120,
    0.95,
    staircase=True)
optimizer = tf.train.AdamOptimizer(
    learning_rate, 0.9).minimize(
    loss, global_step=batch)
train_prediction = tf.nn.softmax(final_fully_connected)